In [2]:
# from MoleculeACE import MPNN, Data, calc_rmse, calc_cliff_rmse, get_benchmark_config
# from Feature.const import Descriptors
# import torch
import os
# import torch.nn as nn
import numpy as np
import pandas as pd
# from main import set_up

In [10]:
from argparse import Namespace
args = Namespace(activation='ReLU', atom_messages=False, batch_size=256, bias=False,
                 checkpoint_dir=None,
                 checkpoint_path='KANO_model/dumped/pretrained_graph_encoder/original_CMPN_0623_1350_14000th_epoch.pkl', checkpoint_paths=['KANO_model/dumped/pretrained_graph_encoder/original_CMPN_0623_1350_14000th_epoch.pkl'],
                 config_path=None, crossval_index_dir=None, crossval_index_file=None, cuda=True,
                 data_path='data/MoleculeACE/CHEMBL1862_Ki.csv', dataset_type='regression', depth=3,
                 dropout=0.0, dump_path='dumped', encoder_name='CMPNN', ensemble_size=1, epochs=100,
                 exp_id='bbbp_test', exp_name='finetune', features_generator=None, features_only=False,
                 features_path=None, features_scaling=True, ffn_hidden_size=300, ffn_num_layers=2, final_lr=0.0001,
                 folds_file=None, gpu=0, hidden_size=300, init_lr=0.0001, log_frequency=10, max_data_size=None,
                 max_lr=0.001, metric='r2', minimize_score=True, multiclass_num_classes=3, no_cache=False,
                 num_lrs=1, num_runs=1, quiet=False, save_dir=None, save_smiles_splits=False, seed=0,
                 separate_test_features_path=None, separate_test_path=None, separate_val_features_path=None,
                 separate_val_path=None, show_individual_scores=False, split_sizes=[0.8, 0.1, 0.1],
                 split_type='scaffold_balanced', step='functional_prompt', temperature=0.1, test=False,
                 test_fold_index=None, undirected=False, use_compound_names=False, use_input_features=None,
                 val_fold_index=None, warmup_epochs=2.0)
args.save_path = 'exp_results/MolTrans'
args.baseline_model = 'MolTrans'
args.mode = 'baseline_CPI'
args.data_name = args.data_path.split('/')[-1].split('.')[0]
args, logger = set_up(args)

2023-10-23 23:48:22,733 - INFO - current task: CHEMBL1862_Ki
2023-10-23 23:48:22,733 - INFO - current task: CHEMBL1862_Ki
2023-10-23 23:48:22,734 - INFO - arguments: Namespace(activation='ReLU', atom_messages=False, baseline_model='MolTrans', batch_size=256, bias=False, checkpoint_dir=None, checkpoint_path='KANO_model/dumped/pretrained_graph_encoder/original_CMPN_0623_1350_14000th_epoch.pkl', checkpoint_paths=['KANO_model/dumped/pretrained_graph_encoder/original_CMPN_0623_1350_14000th_epoch.pkl'], config_path=None, crossval_index_dir=None, crossval_index_file=None, cuda=True, data_name='CHEMBL1862_Ki', data_path='data/MoleculeACE/CHEMBL1862_Ki.csv', dataset_type='regression', depth=3, dropout=0.0, dump_path='dumped', encoder_name='CMPNN', ensemble_size=1, epochs=100, exp_id='bbbp_test', exp_name='finetune', features_generator=None, features_only=False, features_path=None, features_scaling=True, ffn_hidden_size=300, ffn_num_layers=2, final_lr=0.0001, folds_file=None, gpu=0, hidden_size=

In [11]:
MOLECULEACE_DATALIST = ['CHEMBL1862_Ki', 'CHEMBL1871_Ki', 'CHEMBL2034_Ki', 'CHEMBL2047_EC50',
                        'CHEMBL204_Ki', 'CHEMBL2147_Ki', 'CHEMBL214_Ki', 'CHEMBL218_EC50',
                        'CHEMBL219_Ki', 'CHEMBL228_Ki', 'CHEMBL231_Ki', 'CHEMBL233_Ki',
                        'CHEMBL234_Ki', 'CHEMBL235_EC50', 'CHEMBL236_Ki', 'CHEMBL237_EC50',
                        'CHEMBL237_Ki', 'CHEMBL238_Ki', 'CHEMBL239_EC50', 'CHEMBL244_Ki',
                        'CHEMBL262_Ki', 'CHEMBL264_Ki', 'CHEMBL2835_Ki', 'CHEMBL287_Ki',
                        'CHEMBL2971_Ki', 'CHEMBL3979_EC50', 'CHEMBL4005_Ki', 'CHEMBL4203_Ki',
                        'CHEMBL4616_EC50', 'CHEMBL4792_Ki']

In [3]:
from CPI_baseline.MolTrans import MolTrans
from CPI_baseline.utils import MolTrans_config_DBPE, MolTrans_Data_Encoder
from data_prep import process_data_CPI

ModuleNotFoundError: No module named 'torch'

In [13]:
config = MolTrans_config_DBPE()
args.gpu = 0
model = MolTrans(args, logger, config)
if args.gpu is not None:
    device = torch.device("cuda:%d" % args.gpu)
    logger.info("Using GPU: %d" % args.gpu)
else:
    device = torch.device("cpu")
    logger.info("Using CPU")


2023-10-23 23:48:23,221 - INFO - Using GPU: 0
2023-10-23 23:48:23,221 - INFO - Using GPU: 0
2023-10-23 23:48:23,863 - INFO - Batch size 50, Learning rate 0.000100, Num epochs 50
2023-10-23 23:48:23,863 - INFO - Batch size 50, Learning rate 0.000100, Num epochs 50
2023-10-23 23:48:23,863 - INFO - Using GPU: 0
2023-10-23 23:48:23,863 - INFO - Using GPU: 0


In [ ]:
device = torch.device("cuda:%d" % args.gpu)
for i, (d, p, d_mask, p_mask, label) in enumerate(train_data):
    print(d.shape)
    break

torch.Size([50, 50])


In [ ]:
import torch
import esm
import tqdm
import pickle

# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results
token_representations = []
for idx, data in tqdm.tqdm(enumerate(prot_data),
                           total=len(prot_data)):
    batch_labels, batch_strs, batch_tokens = batch_converter([data])
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)
    prot_data[idx] = prot_data[idx] + \
                    (results["representations"][33][0, 1: batch_lens-1].cpu().numpy(),)
    # token_representations.append(results["representations"][33])
with open('data/MoleculeACE/prot_feat_esm2.pkl', 'wb') as f:
    pickle.dump(prot_data, f)

In [5]:
pred = pd.read_csv('exp_results/MolTrans/MolTrans_test_pred.csv')
task = pred['Chembl_id'].unique()
rmse, rmse_cliff = [], []
for i in task:
    pre = pred[pred['Chembl_id']==i]
    label, preds = pre['y'].values, pre['Prediction'].values
    rmse.append(calc_rmse(label, preds))
    rmse_cliff.append(calc_cliff_rmse(preds, label, cliff_mols_test=pre['cliff_mol'].values))
print('RMSE:', np.mean(rmse))
print('RMSE std:', np.std(rmse))
print('RMSE_cliff:', np.mean(rmse_cliff))
print('RMSE_cliff std:', np.std(rmse_cliff))

NameError: name 'calc_rmse' is not defined

In [ ]:
rmse, rmse_cliff = [], []
for task in MOLECULEACE_DATALIST:
    pred = pd.read_csv(f'exp_results/{task}/0/test_pred.csv')
    label, preds = pred['y'].values, pred['Prediction'].values
    rmse.append(calc_rmse(label, preds))
    rmse_cliff.append(calc_cliff_rmse(preds, label, cliff_mols_test=pred['cliff_mol'].values))
print('RMSE:', np.mean(rmse))
print('RMSE std:', np.std(rmse))
print('RMSE_cliff:', np.mean(rmse_cliff))
print('RMSE_cliff std:', np.std(rmse_cliff))

RMSE: 0.7105849246736813
RMSE std: 0.09509675307764494
RMSE_cliff: 0.7872809917927022
RMSE_cliff std: 0.10772807741169514


In [ ]:
import os
from MoleculeACE.benchmark.utils import calc_rmse, calc_cliff_rmse

In [ ]:
rmse, rmse_cliff = [], []
for task in os.listdir(path='exp_results'):
    pred = pd.read_csv(f'exp_results/{task}/0/test_pred.csv')
    label, preds = pred['y'].values, pred['Prediction'].values
    rmse.append(calc_rmse(label, preds))
    rmse_cliff.append(calc_cliff_rmse(label, preds, cliff_mols_test=pred['cliff_mol'].values))

In [ ]:
np.mean(rmse), np.mean(rmse_cliff)

(0.7105849246736813, 0.7872809917927022)

In [ ]:
n_iter = 0
args.epochs = 2
args.prompt = False
metric_dict = set_collect_metric(args)
best_score = float('inf') if args.minimize_score else -float('inf')

for epoch in range(args.epochs):
    print('Current epoch: ', epoch)
    n_iter, loss = train_epoch(args, model, train_data, loss_func, optimizer, scheduler, n_iter)

    if isinstance(scheduler, ExponentialLR):
        scheduler.step()
    val_scores = evaluate_epoch(args, model, val_data, scaler)

    test_pred = predict_epoch(args, model, test_data, scaler)
    test_scores = evaluate_predictions(test_pred, test_data.targets(),
                                       args.num_tasks, args.metric_func, args.dataset_type)
    
    print('Epoch : {:02d}, Training Loss : {:.4f}, ' \
          'Validation score : {:.4f}, Test score : {:.4f}'.format(epoch, loss,
                                 list(val_scores.values())[0][0], list(test_scores.values())[0][0]))
    metric_dict = collect_metric_epoch(metric_dict, loss, val_scores, test_scores)
    
    if args.minimize_score and list(val_scores.values())[0][0] < best_score or \
            not args.minimize_score and list(val_scores.values())[0][0] > best_score:
        best_score, best_epoch = list(val_scores.values())[0][0], epoch
        best_test_score = list(test_scores.values())[0][0]
        save_checkpoint(os.path.join(args.save_dir, 'model.pt'), model, scaler, features_scaler, args) 
        print('Best model saved at epoch : {:02d}, Validation score : {:.4f}'.format(best_epoch, best_score))
print('Final best performed model in {} epoch, val score: {:.4f}, test score: {:.4f}'.format(best_epoch, best_score, best_test_score))

# save results
pickle.dump(metric_dict, open(os.path.join(args.save_dir, 'metric_dict.pkl'), 'wb'))
df['Prediction'] = None
df.loc[test_idx, 'Prediction'] = test_pred
df[df['split']=='test'].to_csv(os.path.join(args.save_dir, 'test_pred.csv'), index=False)

Current epoch:  0
Epoch : 00, Training Loss : 2.0623, Validation score : 1.4718, Test score : 1.4785
Best model saved at epoch : 00, Validation score : 1.4718
Current epoch:  1
Epoch : 01, Training Loss : 2.0272, Validation score : 2.2417, Test score : 1.6129
